### Take a list of images (jpg) + yolo labels and apply Albumentations augmentations

to generate new images correctly labeles (with BB in Yolo v5 format)

In [1]:
from os import path
import glob
import cv2
import albumentations as A
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# read image yolo labels from yolo file
# and returns in yolo format
def read_bb(label_path):
    with open(label_path, 'r') as f:
        data = f.readlines()
    
    # build the bb list
    yolo_bb_list = []
    
    for line in data:
        # Split string to float
        class_num, x, y, w, h = map(float, line.split(' '))
        
        # in the list, class_num is the last, ready for Albumentations
        yolo_bb_list.append([x, y, w, h, int(class_num)])
    
    return yolo_bb_list

# write in a file the transformed bb
def write_bb(new_label_path, yolo_bb_list):
    # a single component in yolo_bb_list is xc, yc, w, h, class_num
    # so we need to change the order in the file (where class_num is the first)
    with open(new_label_path, 'w') as f:
        for bb in yolo_bb_list:
            # get a row
            xc, yc, w, h, class_num = bb
            
            # of decimal digits: 8
            new_line = f"{class_num} {xc:.8f} {yc:.8f} {w:.8f} {h:.8f}\n"
            
            f.write(new_line)
    
# convert a single bb for cv2
def yolo_to_cv2(yolo_bb, height, width):
    # yolo_bb is list o a tuple
    
    # the last is the class_num, here not used
    x, y, w, h, _ = yolo_bb
    
    # x lower left
    l = int((x - w / 2.) * width)
    # x upper right
    r = int((x + w / 2.) * width)
    # y lower left
    t = int((y - h / 2.) * height)
    # y upper right
    b = int((y + h / 2.) * height)
    
    l = max(0, l)
    t = max(0, t)
    r = min(r, width - 1)
    b = min(b, height - 1)
        
    return [l, r, t, b]

# some formal check
def do_check(original_bb_list, trasformed_list):
    # 1. check that # of BB is the same
    # 2. check that, in order, class num is the same
    
    assert len(original_bb_list) == len(trasformed_list), "The two list of BB have not the same lenght"
    
    for o_bb, t_bb in zip(original_bb_list, trasformed_list):
        assert len(o_bb) == len(t_bb), "single BB are not of the same length"
        
        assert o_bb[4] == t_bb[4], "Class num. has changed"
    
    # check the last one, to be used only
    # for my use case
    assert trasformed_list[-1][4] == 10
    
    return

In [6]:
DATASET_BASE_DIR = "/Users/lsaetta/Progetti/yolo_augmentation/yolo_dataset/train"
# under this dir I have images and labels

IMAGE_DEST_DIR = "/Users/lsaetta/Progetti/yolo_augmentation/yolo_dataset/augmented_train/images"
LABELS_DEST_DIR = "/Users/lsaetta/Progetti/yolo_augmentation/yolo_dataset/augmented_train/labels"

images_list = sorted(glob.glob(path.join(DATASET_BASE_DIR, "images/*.jpg")))
labels_list = sorted(glob.glob(path.join(DATASET_BASE_DIR, "labels/*.txt")))

In [7]:
# process in loop
for img_path_name, labels_path_name in zip(images_list, labels_list):
    # remove path
    img_only_name = img_path_name.split("/")[-1]
    labels_only_name = labels_path_name.split("/")[-1]
    
    print(f"Processing {img_only_name}, {labels_only_name}")

Processing 00222_clear_jpg.rf.83b62755045b4f2824f818dfbc82c203.jpg, 00222_clear_jpg.rf.83b62755045b4f2824f818dfbc82c203.txt
Processing 00341_clear_jpg.rf.76dba9fbbeacab4f50d10fe345d19cb1.jpg, 00341_clear_jpg.rf.76dba9fbbeacab4f50d10fe345d19cb1.txt
Processing 00341_clear_rotated_jpg.rf.fbd4206707142423476347f004e7926f.jpg, 00341_clear_rotated_jpg.rf.fbd4206707142423476347f004e7926f.txt
Processing 00597_clear_jpg.rf.9d0d995e279a95e49b2585762aedae12.jpg, 00597_clear_jpg.rf.9d0d995e279a95e49b2585762aedae12.txt
Processing 01643_clear_jpg.rf.4d24f9bfc7096bf9c90e71df0395e1c5.jpg, 01643_clear_jpg.rf.4d24f9bfc7096bf9c90e71df0395e1c5.txt
Processing 02688_clear_jpg.rf.3835244c02d5f89565e9f1dc7c4d93bf.jpg, 02688_clear_jpg.rf.3835244c02d5f89565e9f1dc7c4d93bf.txt
